In [0]:
from pyspark.sql import functions as F, Window as W

dbutils.widgets.text("catalog", "ptd_dev")
catalog       = dbutils.widgets.get("catalog")
spark.sql(f"USE CATALOG {catalog}")

prices = spark.table("silver.prices_daily")
news = spark.table("silver.ticker_news_tone").select("ticker", "date", "weighted_gdelt_tone_avg", "news_count")
try:
    gold = spark.table("gold.ticker_predictor_training_dataset")
except:
    gold = None


In [0]:
if gold is not None:
    last_valid = (gold
        .filter(F.col("adj_close").isNotNull() & F.col("weighted_gdelt_tone_avg").isNotNull())
        .groupBy("ticker")
        .agg(F.max("date").alias("last_valid_date"))
    )
    # Filtrar precios y noticias desde la última fecha válida + 1
    prices = prices.join(last_valid, on="ticker", how="left") \
        .filter((F.col("last_valid_date").isNull()) | (F.col("date") > F.col("last_valid_date"))) \
        .drop("last_valid_date")
    news = news.join(last_valid, on="ticker", how="left") \
        .filter((F.col("last_valid_date").isNull()) | (F.col("date") > F.col("last_valid_date"))) \
        .drop("last_valid_date")

In [0]:
last_price_date = prices.groupBy("ticker").agg(F.max("date").alias("last_date"))

news = news.join(last_price_date, on="ticker", how="inner").filter(F.col("date") <= F.col("last_date")).drop("last_date")

w = W.partitionBy("ticker").orderBy("date")
prices = (prices
    .withColumn("adj_close_next", F.lead("adj_close", 1).over(w))
    .withColumn("target_action",
        F.when(F.col("adj_close_next") > F.col("adj_close"), F.lit("buy"))
         .when(F.col("adj_close_next") < F.col("adj_close"), F.lit("sell"))
         .otherwise(F.lit("hold"))
    )
)

In [0]:
dataset = prices.join(news, on=["ticker", "date"], how="left")
dataset = dataset.filter(F.col("ticker").isNotNull() & F.col("date").isNotNull())

table_name = "gold.ticker_predictor_training_dataset"
existing = None
try:
    existing = spark.table(table_name)
except Exception:
    existing = None

if existing is not None:

    joined = dataset.join(
        existing.select("ticker", "date", "adj_close", "weighted_gdelt_tone_avg"),
        ["ticker", "date"],
        "left"
    )
    # Registros nuevos: no existen en la tabla destino
    new_rows = joined.filter(F.col("adj_close_y").isNull())
    # Registros modificados: existen pero algún campo relevante cambió
    modified_rows = joined.filter(
        (F.col("adj_close_y").isNotNull()) & (
            (F.col("adj_close") != F.col("adj_close_y")) |
            (F.col("weighted_gdelt_tone_avg") != F.col("weighted_gdelt_tone_avg_y"))
        )
    )
    print(f"Registros nuevos: {new_rows.count()}")
    print(f"Registros modificados: {modified_rows.count()}")
else:
    print(f"Registros nuevos: {dataset.count()}")
    print("Registros modificados: 0")

# Guardar el resultado
dataset.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable(table_name)